# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [106]:
file_path = "../WeatherPy/output_data/cities.csv"

cities_df = pd.read_csv(file_path)
cities_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,cabo san lucas,22.8909,-109.9124,75.20,53,1,8.05,MX,1611878592
1,hobart,-42.8794,147.3294,64.99,55,75,13.80,AU,1611878721
2,roald,62.5833,6.1333,33.80,64,75,17.27,NO,1611878940
3,cape town,-33.9258,18.4232,63.00,82,0,4.61,ZA,1611878888
4,avare,-23.0986,-48.9258,70.20,90,98,2.95,BR,1611878774
...,...,...,...,...,...,...,...,...,...
541,westport,41.1415,-73.3579,25.00,42,1,11.50,US,1611878893
542,mogadishu,2.0371,45.3438,76.48,79,0,19.26,SO,1611879060
543,boa vista,2.8197,-60.6733,81.28,54,34,12.41,BR,1611878837
544,kieta,-6.2167,155.6333,85.39,72,100,12.95,PG,1611879035


In [107]:
import requests
import json


target_city = "Boise, Idaho"

target_url = ('https://maps.googleapis.com/maps/api/geocode/json?'
    'address={0}&key={1}').format(target_city, g_key)

response = requests.get(target_url)
print(response)

json_response = response.json()
print(json_response)

<Response [200]>
{'results': [{'address_components': [{'long_name': 'Boise', 'short_name': 'Boise', 'types': ['locality', 'political']}, {'long_name': 'Ada County', 'short_name': 'Ada County', 'types': ['administrative_area_level_2', 'political']}, {'long_name': 'Idaho', 'short_name': 'ID', 'types': ['administrative_area_level_1', 'political']}, {'long_name': 'United States', 'short_name': 'US', 'types': ['country', 'political']}], 'formatted_address': 'Boise, ID, USA', 'geometry': {'bounds': {'northeast': {'lat': 43.6898951, 'lng': -116.1019091}, 'southwest': {'lat': 43.511717, 'lng': -116.3658869}}, 'location': {'lat': 43.6150186, 'lng': -116.2023137}, 'location_type': 'APPROXIMATE', 'viewport': {'northeast': {'lat': 43.6898951, 'lng': -116.1019091}, 'southwest': {'lat': 43.511717, 'lng': -116.3658869}}}, 'place_id': 'ChIJnbRH6XLxrlQRm51nNpuYW5o', 'types': ['locality', 'political']}], 'status': 'OK'}


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [108]:
gmaps.configure(api_key=g_key)

In [109]:
locations = cities_df[['Lat','Lng']]
locations

,Lat,Lng
0,22.8909,-109.9124
1,-42.8794,147.3294
2,62.5833,6.1333
3,-33.9258,18.4232
4,-23.0986,-48.9258
...,...,...
541,41.1415,-73.3579
542,2.0371,45.3438
543,2.8197,-60.6733
544,-6.2167,155.6333


In [110]:
humidity_cities = cities_df['Humidity']
humidity_cities
# cities_df['Humidity'].max()

0      53
1      55
2      64
3      82
4      90
       ..
541    42
542    79
543    54
544    72
545    83
Name: Humidity, Length: 546, dtype: int64

In [111]:
# Plot Heatmap
fig = gmaps.figure(center=(0,0),zoom_level=2)

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity_cities, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=2, opacity=0.6)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [112]:
cities_df.dtypes

City           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
Country        object
Date            int64
dtype: object

In [113]:
# A max temperature lower than 80 degrees but higher than 70.
# Wind speed less than 10 mph.
# Zero cloudiness.
# Drop any rows that don’t contain all three conditions. You want to be sure the weather is ideal.

weather_fit_df = cities_df.loc[(cities_df['Max Temp'] < 80) & (cities_df['Max Temp'] >= 70) & (cities_df['Wind Speed'] < 10) & (cities_df['Cloudiness'] == 0)]

weather_fit_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
13,beterou,9.1992,2.2586,74.93,20,0,4.21,BJ,1611878942
53,caravelas,-17.7125,-39.2481,76.46,83,0,6.24,BR,1611878902
92,santiago de cao,-7.9589,-79.2392,71.60,78,0,8.05,PE,1611878958
183,torit,4.4133,32.5678,78.66,33,0,2.73,SS,1611878977
229,yendi,9.4427,-0.0099,78.28,17,0,4.07,GH,1611878988
288,monrovia,6.3005,-10.7969,77.00,94,0,8.48,LR,1611879005
315,dagana,16.5165,-15.5068,75.20,23,0,2.30,MR,1611879011
378,sao filipe,14.8961,-24.4956,73.29,66,0,5.35,CV,1611878649
402,bobo dioulasso,11.1771,-4.2979,71.73,19,0,3.89,BF,1611879030
422,warmbad,-28.4500,18.7333,73.76,21,0,9.48,NaN,1611879034


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
